In [45]:
using LinearAlgebra
using Plots
using Statistics
using Random
using ForwardDiff
using Optim

In [83]:
# Parameters
alpha_1 = 0.9
alpha_2 = -0.2
beta = 0.8
sigma_1 = 1.0
sigma_2 = 1.0
sigma_12 = 0.5

# Initial conditions (shouldn't matter too much)
x_0 = 0.0
y_0 = 0.0

# DGP conditions
T = 1000000
Random.seed!(1234)

# Generate data
X = x_0 .* ones(T) .* 0
Y = y_0 .* ones(T) .* 0
W = randn(T,2)

# Spot check
println("mean: ", mean(W))
println("var: ", var(W))

# Build in a detailed but bad way
for t in 1:T-1
    Y[t+1] = alpha_1*Y[t] + alpha_2*X[t] + sigma_1*W[t+1,1] + sigma_12*W[t+1,2]
    X[t+1] = beta*X[t] + sigma_2*W[t+1,2]
end

# With matrices
Z = [y_0 .* ones(T) x_0 .* ones(T)]

# Parameters
A = [alpha_1 alpha_2 ; 0 beta]
B = [sigma_1 sigma_12 ; 0 sigma_2]

# Build in a simple, good way
for t in 1:T-1
    Z[t+1,:] = A*Z[t,:] + B*W[t+1,:]
end


mean: 0.0008073276783183449
var: 0.9990963717486129


In [84]:
# Sanity check for data construction
# pX = plot(X)
# plot!(Z[:,2])
# display(pX)

# pY = plot(Y)
# plot!(Z[:,1])
# display(pY)

In [85]:
# Calculate log likelihood
function log_lik_mat(A,B,Z)
    return -1/2 * sum([(B \ (Z[t+1,:] - A*Z[t,:]))'*(B \ (Z[t+1,:] - A*Z[t,:])) for t in 1:size(Z,1)-1])
end

# negative of log-lik, as function of alpha_1 only
simp_mat(a) = -log_lik_mat([a alpha_2; 0 beta], [sigma_1 sigma_12; 0 sigma_2], Z)
println("simp_mat: ", simp_mat(alpha_1))

# Estimate alpha_1_hat by MLE
alpha_1_hat = optimize(simp_mat, [0.0], BFGS(), Optim.Options(show_trace = true); autodiff = :forward).minimizer[1]
println("alpha_1_hat: ", alpha_1_hat)

# Calculate gradient and Hessian of loglik
grad_simp_mat(a) = ForwardDiff.gradient(simp_mat, [a])
hess_simp_mat(a) = ForwardDiff.hessian(simp_mat, [a])
# println("grad_mat: ", grad_simp_mat(alpha_1))
# println("FI simp mat grad: ", (grad_simp_mat(alpha_1)'*grad_simp_mat(alpha_1)))

# Fisher Info (normalize by T)
println("FI simp mat hess: ", (1/T * hess_simp_mat(alpha_1_hat)))

println()

# negative of loglik, as function of alpha_1, alpha_2, beta
full_mat(theta) = -log_lik_mat([theta[1] theta[2]; 0 theta[3]], [sigma_1 sigma_12; 0 sigma_2], Z)
theta_0 = [alpha_1 alpha_2 beta]
println("full_mat: ", full_mat(theta_0))

# Estimate theta by MLE
theta_hat = optimize(full_mat, [0.0, 0.0, 0.0], BFGS(), Optim.Options(show_trace = true); autodiff = :forward).minimizer
println("theta_hat: ", theta_hat)

# Get grad and Hess of loglik
grad_full_mat(theta) = ForwardDiff.gradient(full_mat, theta)
hess_full_mat(theta) = ForwardDiff.hessian(full_mat, theta)

# "Fisher Info" (normalize by T)
println("FI for alpha_1: ", 1/((1/T * hess_full_mat(theta_hat) \ I)[1,1]))

# We ought to have less info when we have nuisances (it seems this holds)

simp_mat: 999095.9528099515
Iter     Function value   Gradient norm 
     0     3.758276e+06     6.128020e+06
 * time: 0.00030493736267089844
     1     9.990951e+05     9.214318e-10
 * time: 3.356060028076172
alpha_1_hat: 0.900512988889434
FI simp mat hess: [6.805032374304787]

full_mat: 999095.9528099515
Iter     Function value   Gradient norm 
     0     5.032770e+06     6.005163e+06
 * time: 0.0002880096435546875
     1     1.181045e+06     1.041502e+06
 * time: 2.9404799938201904
     2     1.017153e+06     2.396884e+05
 * time: 5.616627931594849
     3     9.990949e+05     6.046434e-10
 * time: 8.112473964691162
theta_hat: [0.9005077387073848, -0.19997570010958343, 0.7996996321071902]
FI for alpha_1: 6.789907946129881


In [ ]:
# Discarded Code that I wanted to hold onto (just in case)

# function log_lik(theta, sigma, X, Y)
#     a1 = theta[1]
#     a2 = theta[2]
#     b = theta[3]
    
#     s1 = sigma[1]
#     s2 = sigma[2]
#     s12 = sigma[3]
    
#     T = length(Y)
# #     println("size: ", size((1/s1 .* (Y[2:T] .- a1 .* Y[1:T-1] .- a2 .* X[1:T-1] .- s12/s2 .* (X[2:T] .- b .* X[1:T-1]))).^ 2))
    
#     return -1/2*sum((1/s1 .* (Y[2:T] - a1 .* Y[1:T-1] - a2 .* X[1:T-1] - s12/s2 .* (X[2:T] - b .* X[1:T-1]))).^ 2)
# end

# println("full: ", log_lik([alpha_1 alpha_2 beta], [sigma_1 sigma_2 sigma_12], X, Y))

# simp_ll(a) = log_lik([a alpha_2 beta], [sigma_1 sigma_2 sigma_12], X, Y)
# println("simp_ll: ", simp_ll(alpha_1))
# grad_simp_ll(a) = ForwardDiff.gradient(simp_ll, [a])
# hess_simp_ll(a) = ForwardDiff.hessian(simp_ll, [a])

# println("grad: ", grad_simp_ll(alpha_1))
# println("FI simp grad: ", (grad_simp_ll(alpha_1)'*grad_simp_ll(alpha_1)) \ I)
# println("FI simp hess: ", (-hess_simp_ll(alpha_1)) \ I)
# println()

# full_ll(theta) = log_lik(theta, [sigma_1 sigma_2 sigma_12], X, Y)
# grad_full_ll(theta) = ForwardDiff.gradient(full_ll, theta)
# hess_full_ll(theta) = ForwardDiff.hessian(full_ll, theta)

# theta_0 = [alpha_1 alpha_2 beta]

# println("full_ll: ", full_ll(theta_0))
# println("grad_full test: ", grad_full_ll(theta_0))
# println("grad_full_ll: ", (grad_full_ll(theta_0)'*grad_full_ll(theta_0)))
# println("FI full grad: ", (grad_full_ll(theta_0)'*grad_full_ll(theta_0)))
# println("FI full hess: ", (-hess_full_ll(theta_0)))